In [40]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [41]:
CLIENT_ID = 'XWWKC4DCATGXLDLXCE2F3UATKKBSNBGEF2D4CCXWK1CRJDDH' # your Foursquare ID
CLIENT_SECRET = 'ADWIXYOQ3SH5SRZZLNVJJQK3FCKBHPEBJDJUT3VNJ54E3WGT' # your Foursquare Secret
VERSION = '20200215'
LIMIT = 100

In [42]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [43]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [44]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [45]:
neighborhoods_data = newyork_data['features']
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [46]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [47]:
neighborhoods.shape

(306, 4)

In [48]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [49]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Manhattan using latitude and longitude values
map_bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

In [50]:
bronx_data.loc[0, 'Neighborhood']
neighborhood_latitude = bronx_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = bronx_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = bronx_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [51]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XWWKC4DCATGXLDLXCE2F3UATKKBSNBGEF2D4CCXWK1CRJDDH&client_secret=ADWIXYOQ3SH5SRZZLNVJJQK3FCKBHPEBJDJUT3VNJ54E3WGT&v=20200215&ll=40.89470517661,-73.84720052054902&radius=1000&limit=100'

In [52]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e47d82eb1cac0001bd517de'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Wakefield',
  'headerFullLocation': 'Wakefield, Bronx',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 40.903705185610015,
    'lng': -73.83531662200086},
   'sw': {'lat': 40.88570516760999, 'lng': -73.85908441909719}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c537892fd2ea593cb077a28',
       'name': 'Lollipops Gelato',
       'location': {'address': '4120 Baychester Ave',
        'crossStreet': 'Edenwald & Bussing Ave',
        'lat': 40.894123150205274,
        'ln

In [53]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [54]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lollipops Gelato,Dessert Shop,40.894123,-73.845892
1,Ripe Kitchen & Bar,Caribbean Restaurant,40.898152,-73.838875
2,Ali's Roti Shop,Caribbean Restaurant,40.894036,-73.856935
3,Jackie's West Indian Bakery,Caribbean Restaurant,40.889283,-73.843310
4,Carvel Ice Cream,Ice Cream Shop,40.890487,-73.848568


In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [58]:
print(bronx_venues.shape)
bronx_venues.head()

(3268, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop


In [59]:
bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,73,73,73,73,73,73
Baychester,100,100,100,100,100,100
Bedford Park,76,76,76,76,76,76
Belmont,100,100,100,100,100,100
Bronxdale,100,100,100,100,100,100
Castle Hill,44,44,44,44,44,44
City Island,41,41,41,41,41,41
Claremont Village,17,17,17,17,17,17
Clason Point,7,7,7,7,7,7


In [60]:
print('There are {} uniques categories.'.format(len(bronx_venues['Venue Category'].unique())))

There are 225 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Coffee shop
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Coffee shop
2,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Coffee shop
3,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Coffee shop
4,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Coffee shop


(3268, 7)

In [ ]:
#Try to clusterneighbourhood in Bronx

In [61]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

# set number of clusters
kclusters = 5

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bronx_merged = bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head() # check the last columns!

----Allerton----
----Baychester----
----Bedford Park----
----Belmont----
----Bronxdale----
----Castle Hill----
----City Island----
----Claremont Village----
----Clason Point----
----Co-op City----
----Concourse----
----Concourse Village----
----Country Club----
----East Tremont----
----Eastchester----
----Edenwald----
----Edgewater Park----
----Fieldston----
----Fordham----
----High  Bridge----
----Hunts Point----
----Kingsbridge----
----Kingsbridge Heights----
----Longwood----
----Melrose----
----Morris Heights----
----Morris Park----
----Morrisania----
----Mott Haven----
----Mount Eden----
----Mount Hope----
----North Riverdale----
----Norwood----
----Olinville----
----Parkchester----
----Pelham Bay----
----Pelham Gardens----
----Pelham Parkway----
----Port Morris----
----Riverdale----
----Schuylerville----
----Soundview----
----Spuyten Duyvil----
----Throgs Neck----
----Unionport----
----University Heights----
----Van Nest----
----Wakefield----
----West Farms----
----Westchester Squ

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0,Pharmacy,Supermarket,Caribbean Restaurant,Donut Shop,Gas Station,Fast Food Restaurant,Bus Station,Fried Chicken Joint,Bank,Chinese Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,0,Pizza Place,Department Store,Mobile Phone Shop,Shopping Mall,Bakery,Bus Station,Chinese Restaurant,Furniture / Home Store,Shoe Store,Pharmacy
2,Bronx,Eastchester,40.887556,-73.827806,0,Caribbean Restaurant,Shopping Mall,Diner,Grocery Store,Fast Food Restaurant,Asian Restaurant,Gas Station,Mobile Phone Shop,Burger Joint,Bus Station
3,Bronx,Fieldston,40.895437,-73.905643,1,Pizza Place,Deli / Bodega,Bar,Playground,Mexican Restaurant,Plaza,Sandwich Place,Park,Bank,Bus Station
4,Bronx,Riverdale,40.890834,-73.912585,1,Pizza Place,Bar,Bank,Park,Deli / Bodega,Mexican Restaurant,Japanese Restaurant,Plaza,Bagel Shop,Coffee Shop


In [62]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 0, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Supermarket,Caribbean Restaurant,Donut Shop,Gas Station,Fast Food Restaurant,Bus Station,Fried Chicken Joint,Bank,Chinese Restaurant
1,Co-op City,Pizza Place,Department Store,Mobile Phone Shop,Shopping Mall,Bakery,Bus Station,Chinese Restaurant,Furniture / Home Store,Shoe Store,Pharmacy
2,Eastchester,Caribbean Restaurant,Shopping Mall,Diner,Grocery Store,Fast Food Restaurant,Asian Restaurant,Gas Station,Mobile Phone Shop,Burger Joint,Bus Station
7,Norwood,Pizza Place,Supermarket,Donut Shop,Bank,Mexican Restaurant,Sandwich Place,Pharmacy,Park,Gym,Fast Food Restaurant
8,Williamsbridge,Pizza Place,Caribbean Restaurant,Sandwich Place,Supermarket,Pharmacy,Seafood Restaurant,Bus Station,Bakery,Fried Chicken Joint,Cosmetics Shop
9,Baychester,Clothing Store,Bus Station,Department Store,Pizza Place,Pharmacy,Accessories Store,Shopping Mall,Sandwich Place,Supermarket,Discount Store
10,Pelham Parkway,Pizza Place,Chinese Restaurant,Deli / Bodega,Donut Shop,Italian Restaurant,Sandwich Place,Bank,Bakery,Supermarket,Cosmetics Shop
12,Bedford Park,Pizza Place,Park,Supermarket,Diner,Sandwich Place,Donut Shop,Deli / Bodega,Baseball Field,Café,Bank
13,University Heights,Grocery Store,Pizza Place,Donut Shop,Supermarket,Deli / Bodega,Spanish Restaurant,Latin American Restaurant,Ice Cream Shop,Fast Food Restaurant,Fried Chicken Joint
14,Morris Heights,Pizza Place,Food Truck,Grocery Store,Pharmacy,Supermarket,Fast Food Restaurant,Intersection,Bus Station,Gym / Fitness Center,Spanish Restaurant


In [ ]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 1, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

In [ ]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

In [ ]:
#set a new dataframe for coffee shop
df_coffee = bronx_venues[bronx_venues['Venue Category']=='Coffee shop']
df_coffee.shape